# Part 2, Topic 1: CPA Attack on 32bit AES (HARDWARE)

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

Usual capture, just using MBEDTLS instead of TINYAES128

In [4]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32L4' #'CWLITEARM'
#PLATFORM = 'CW308_STM32F4'
#PLATFORM = 'CW308_STM32F3'

CRYPTO_TARGET = "TINYAES128C"#"HWAES", "MBEDTLS", "TINYAES128C"
#CRYPTO_TARGET = None
SS_VER = 'SS_VER_1_1'
#CHECK_CORR = False ##From CW_lite code

In [3]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

(ChipWhisperer NAEUSB WARNING|File naeusb.py:743) Your firmware (0.30) is outdated - latest is 0.65 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information


INFO: Found ChipWhisperer😍
scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 30                       
scope.gain.db                            changed from 5.5                       to 24.8359375               
scope.adc.basic_mode                     changed from low                       to rising_edge              
scope.adc.samples                        changed from 24400                     to 5000                     
scope.adc.trig_count                     changed from 7791757                   to 32770947                 
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.adc_freq                     changed from 0                         to 373863281                
scope.clock.adc_rate                     changed from 0.0                       to 373863281.0       

In [5]:
%%bash
cd ../../../hardware/victims/firmware/
mkdir -p simpleserial-aes-l443_tinyaes && cp -r simpleserial-aes/* $_

In [9]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-aes-l443_tinyaes
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3

Building for platform CW308_STM32L4 with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
Blank crypto options, building for AES128
Building for platform CW308_STM32L4 with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
Blank crypto options, building for AES128
make[1]: '.dep' is up to date.
Building for platform CW308_STM32L4 with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
Blank crypto options, building for AES128
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (15:10.3-2021.07-4) 10.3.1 20210621 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling:
-en     simpleserial-aes.c ...
-e Done!
.
Compiling:
-en     .././simpleserial/simpleserial.c ...
-e Done!
.
Compiling:
-en     .././hal/stm32l4/stm32l4_h

In [10]:
fw_path = '../../../hardware/victims/firmware/simpleserial-aes-l443_tinyaes/simpleserial-aes-{}.hex'.format(PLATFORM)
cw.program_target(scope, prog, fw_path)

Detected unknown STM32F ID: 0x435
Extended erase (0x44), this can take ten seconds or more
Attempting to program 8055 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 8055 bytes


In [11]:
#Capture Traces
from tqdm import tnrange, trange
import numpy as np
import time

ktp = cw.ktp.Basic()
ktp.fixed_key = False

traces = []
N = 100  # Number of traces
project = cw.create_project("traces/checkTraces1.cwp", overwrite=True)

for i in trange(N, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here

    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    project.traces.append(trace)

try:
    print(scope.adc.trig_count) # print if this exists
except:
    pass
project.save()

Capturing traces: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 17.76it/s]

50095


In [12]:
scope.dis()
target.dis()

In [13]:
ktp.fixed_key =False
ktp.next()

(CWbytearray(b'68 c3 46 bf ca 73 7e 12 49 cf e3 d6 5d 84 c8 50'),
 CWbytearray(b'b5 5e db 6e d1 00 ab 0a 43 92 85 96 da d7 21 60'))